In [3]:

from Experiment import Experiment
import matplotlib.pyplot as plt
import Plotters 
import plotly.graph_objects as go
import plotly
import numpy as np

import time
import matplotlib.cm as colormap

%matplotlib qt

loadir = 'H:/My Drive/dark 2022/csv_dark' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name =['manipulated_2022_02_03_dark',
           'manipulated_2023_08_10_100ms',
           'manipulated_2023_08_09_80ms',
           'manipulated_2023_08_09_60ms',
           'manipulated_2023_08_06_40ms',
           'manipulated_2023_08_07_10ms',
           'manipulated_2023_08_07_5ms',
] 

def dont_show_movs(exp_name,movs_to_del):
    return [mov_name for mov_name in experiments[exp_name].mov_names if mov_name not in movs_to_del ] 

mov_dark_delete = ['mov30','mov40','mov57','mov59']
mov_100_delete = ['mov10','mov102','mov19','mov34','mov35','mov47','mov51','mov52','mov66','mov68','mov73','mov90']
mov_80_delete = ['mov119','mov49','mov97','mov99']
mov_40_delete = ['mov64']
mov_60_delete = ['mov12','mov13','mov147','mov165','mov21','mov24','mov26','mov48','mov86']
mov_10_delete = ['mov46','mov54']
mov_5_delete = ['mov44']


color_map = colormap.datad["tab10"]['listed']

experiments = {exp:Experiment(loadir,exp) for exp in exp_name}
[exp.smooth_prop_movies('CM_real_x_body',['','_dot','_dot_dot'],'body')  for exp in experiments.values()]


[exp.project_axes_movies(['X_x_body_projected','X_y_body_projected'])  for exp in experiments.values()] # project X body to xy
[exp.project_axes_movies(['CM_dot_x_xy','CM_dot_y_xy'],axis = 'CM_real_x_body_dot_smth',wing_body = 'body',add_to_vectors = True)  for exp in experiments.values()] # project X body to xy


[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_x',add_to_vectors = True)  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_y',ax_to_proj = 'Y_x_body',add_to_vectors = True)  for exp in experiments.values()]



[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_x')  for exp in experiments.values()]

[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y',ax_to_proj = 'Y_x_body')  for exp in experiments.values()]


[exp.pqr_movies() for exp in experiments.values()]
# [exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi') for exp in experiments]
[exp.substruct_first_frame('yaw_body','body')  for exp in experiments.values()]
[exp.velocity_amplitude('CM_dot_x_projected','CM_dot_y_projected','body')  for exp in experiments.values()]

# calculate the difference between current direction and another one at the same time
[exp.delta_ang_all_time_movies('X_x_body_projected','CM_dot_x_xy','slip_angle_beta',three_col = 2) for exp in experiments.values()]

# calculate the difference between current property and in referance frame
[exp.delta_ref_ang_movies('X_x_body_projected','delta_ang_Xax',axis = 'X_x_body_projected')  for exp in experiments.values()]
[exp.delta_ref_ang_movies('CM_dot_x_projected','delta_ang_CM_dot_x',axis = 'CM_dot_x_projected') for exp in experiments.values()]




Change in axis (delta angle) compared to its projection in t = 0

In [2]:
exp = experiments['manipulated_2023_08_07_5ms']
fig = go.Figure()
exp.plot_prop_movies('delta_ang_CM_dot_x','vectors',fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = None)
fig.show()

fig = go.Figure()
exp.plot_prop_movies('slip_angle_beta','vectors',fig,case = 'plot_mov',prop_x = 'time')
fig.show()
fig = go.Figure()
exp.plot_prop_movies('yaw_z_frame','body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()
fig = go.Figure()
exp.plot_prop_movies('delta_ang_Xax','vectors',fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = None)
fig.show()




In [2]:
fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('CM_dot_dot_y_projected','body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)

fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('phi_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('phi_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)

fig.show()

In [54]:

mov_to_del = [mov_dark_delete,mov_100_delete,mov_80_delete,mov_60_delete,mov_40_delete,mov_10_delete,mov_5_delete]

for idx,(exp,mov_to_del) in enumerate(zip(experiments.values(),mov_to_del)):
    fig = go.Figure()

    exp.plot_prop_movies('delta_ang','vectors',fig,case = 'plot_mov',prop_x = 'time', mov = dont_show_movs(exp.experiment_name,mov_to_del))
    fig.update_layout(title = exp.experiment_name)
    fig.show()

mov_to_del = [mov_dark_delete,mov_100_delete,mov_80_delete,mov_60_delete,mov_40_delete,mov_10_delete,mov_5_delete]

fig = go.Figure()
[exp.plot_prop_movies('delta_ang','vectors',fig,case = 'plot_exp',prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name, 
                      mov = dont_show_movs(exp.experiment_name,mov_to_del)) for idx,(exp,mov_to_del) in enumerate(zip(experiments.values(),mov_to_del))]
fig.show()

3D trajectory plot

In [14]:

exp = experiments['manipulated_2023_08_09_60ms']
mov = ['mov75']
exp.plot_3d_traj_movies('CM_dot_x_projected',mov = mov)


Calculate first peak/ min/ max

In [5]:
fig = go.Figure()
exp = experiments[list(experiments.keys())[0]]
[exp.get_peaks_movies('pitch_y_frame_dot_dot','peaks_max',t0 = 5, t1 = 50) for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame_dot_dot','min') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','peaks_max',t0 = 40, t1 = 100) for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame','max') for exp in experiments.values()]

[exp.get_peaks_movies('CM_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','zero_v') for exp in experiments.values()]

[exp.get_peaks_movies('amp_v','peaks_min') for exp in experiments.values()]


[None, None, None, None, None, None, None]

In [24]:
for exp_name in experiments:
    exp = experiments[exp_name]
    time = exp.interest_points['pitch_y_frame_max'][:,exp.body_header['time']]
    max_pitch = exp.interest_points['pitch_y_frame_max'][:,exp.body_header['pitch_body']]

    plt.plot(time,
            max_pitch,'*',label = exp_name)
plt.legend()

Plot histograms of peaks, plot movies time seires data with the max/min/peak data point on them

In [64]:
# Histograms
point_name = 'pitch_y_frame_dot_dot_peaks_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()

point_name = 'pitch_y_frame_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'amp_v_peaks_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_projected_zero_v'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()

In [9]:
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('CM_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('amp_v_peaks_min','amp_v',fig) 
fig.show()

In [68]:
# time series of movies with min/max on them
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','pitch_y_frame_dot_dot',fig) 
fig.show()

fig = go.Figure()
exp.plot_prop_movies('CM_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','CM_dot_projected',fig) 
fig.show()

In [45]:



# time series of movies with min/max on them
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','pitch_y_frame_dot_dot',fig) 
fig.show()

fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_min','pitch_y_frame',fig) 
fig.show()


fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_max','pitch_y_frame_dot_dot',fig) 
fig.show()


fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_dot_projected_min','CM_dot_dot_projected',fig) 
fig.show()


fig = go.Figure()
exp.plot_prop_movies('yaw_z_frame','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_projected_zero_v','yaw_z_frame',fig) 
fig.show()


# 


In [23]:

fig = go.Figure()
exp.plot_prop_movies('roll_x_frame','body',fig,case = 'plot_mov',prop_x = 'time')
# exp.add_point_to_plot_movies('CM_dot_projected_zero_v','CM_dot_y_projected',fig) 
exp.plot_prop_movies('yaw_z_frame','body',fig,case = 'plot_mov',prop_x = 'time',yaxis = 'y2')
# exp.add_point_to_plot_movies('CM_dot_projected_zero_v','roll_body',fig,yaxis = 'y2') 

# yax_dict[f"yaxis2"] = (dict(title=prop,titlefont=dict(color=color),tickfont=dict(color=color),side = side[i],position=position[i]))



fig.update_layout(title = exp.experiment_name,
    xaxis=dict(domain=[0, 0.95]),
    yaxis=dict(title='roll_x_frame', titlefont=dict(color='red'), tickfont=dict(color='red'), side='left', position=0),
    yaxis2=dict(title='yaw_y_frame', titlefont=dict(color='green'), tickfont=dict(color='green'), side='right', position=0.95),
)

fig.show()

Plot all experiments - time series

In [6]:
fig = go.Figure()
prop = 'pitch_y_frame_dot_dot'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()

Calculate and plot mean property (time series,  mean per frame of all movies)

In [53]:
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'r'
prop = 'CM_dot_dot_projected'
props = ['delta_ang','delta_ang']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'vectors')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean, label = exp.experiment_name)


plt.legend()

g:\Documents\experiment_analysis\data_analysis\Experiment.py:121: RuntimeWarning:

Mean of empty slice



Try PCA

In [19]:
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.cluster import KMeans

mov_feat = []
props = ['CM_dot_dot_projected','CM_dot_projected','pitch_y_frame_dot_dot']

exp_pd = []
for exp in list(experiments.values())[0:3]:
    mov_feat = []
    for idx,mov_name in enumerate(exp.mov_names):
        mov = exp.get_mov(mov_name)
        mov_feat.append(np.nanmean(mov.data['body'][15:60,:],axis = 0))

    featpd_exp = pd.DataFrame(data = mov_feat,columns = mov.header['body'])
    featpd_exp['exp_name'] = exp.experiment_name
    featpd_exp['mov'] = exp.mov_names
    exp_pd.append(featpd_exp)

feat = pd.concat(exp_pd,axis = 0).dropna().reset_index()


dataforpca = feat[props]
n_comp = 2
pca = PCA(n_components=n_comp)
principalComponents = pca.fit_transform(dataforpca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = [f'pca_{idx}' for idx in range(n_comp)])

pca_name = [f'pca_{idx}' for idx in range(pca.n_components_)]


featpd = pd.concat([feat, principalDf], axis = 1)
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(featpd[pca_name])
print(kmeans.labels_)
pca.components_


C:\Users\Roni\AppData\Local\Temp\ipykernel_31184\2630784800.py:13: RuntimeWarning:

Mean of empty slice

C:\Users\Roni\AppData\Local\Temp\ipykernel_31184\2630784800.py:13: RuntimeWarning:

Mean of empty slice

C:\Users\Roni\AppData\Local\Temp\ipykernel_31184\2630784800.py:13: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



[0 0 0 0 0 0 0 2 2 2 0 0 2 1 0 0 1 0 2 2 2 2 2 0 0 2 2 2 0 0 0 2 0 0 2 2 0
 0 2]


array([[ 2.58930919e-03, -3.21593360e-07, -9.99996648e-01],
       [-9.99996645e-01, -7.69076209e-05, -2.58930916e-03]])

In [18]:

fig = go.Figure()

for lbl in np.unique(kmeans.labels_):
    exp_names = np.unique(feat.iloc[kmeans.labels_==lbl]['exp_name'])
    feat_lbl = feat.iloc[kmeans.labels_==lbl]
    for idx,exp_name in enumerate(exp_names): 
        mov_exp_lbl = list(feat_lbl.loc[feat_lbl['exp_name'] == exp_name]['mov'])

        prop = 'pitch_y_frame_dot'
        prop = 'CM_dot_dot_projected'
        # prop = 'pitch_body'

        color_map = colormap.datad["tab10"]['listed']

        experiments[exp_name].plot_prop_movies(prop,'body',fig,mov = mov_exp_lbl,case = 'plot_exp',prop_x = 'time',color = color_map[lbl],legend = f'{exp_name}_cluster{lbl}')
fig.show()
# feat.iloc[kmeans.labels_==lbl]['exp_name'] == exp_names[0]
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'q'
prop = 'CM_dot_dot_projected'
props = ['pitch_y_frame_dot_dot','pitch_y_frame_dot']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'body')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean)

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Experiment.py:112: RuntimeWarning:

Mean of empty slice



Plot and save figures

In [11]:
exp

In [4]:
import plotly.graph_objects as go
import plotly
import matplotlib.cm as colormap
color_map = colormap.datad["tab10"]['listed']



def plot_and_save_movie_prop(exp,yprop,wing_body,title,add_horizontal_line = None):
    
    fig = go.Figure()

    exp.plot_prop_movies(yprop,wing_body,fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = add_horizontal_line)
    fig.update_layout(title = title)
   
    fig.show()
    plotly.offline.plot(fig, filename=f'{exp.figures_path}/{yprop}_plot.html',auto_open=False)




for exp in experiments:
    try:
        exp = experiments[exp]
        exp.plot_3d_traj_movies('CM_dot_x_projected',save_plot = True, mov = False)

        plot_and_save_movie_prop(exp, 'slip_angle_beta', 'vectors','dot product between Xbody_xy and V_xy', add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'delta_ang_CM_dot_x','vectors','dot product between initial V_xy and V_xy',add_horizontal_line = None)
        plot_and_save_movie_prop(exp, 'delta_ang_Xax','vectors','dot product between initial Xbody_xy and Xbody_xy',add_horizontal_line = None)

        plot_and_save_movie_prop(exp, 'CM_dot_dot_x_projected','acceleration projected on XY',wing_body = 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'pitch_y_frame','pitch angle rotated to first frame', 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'pitch_y_frame_dot','pitch_dot rotated to first frame' 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'CM_dot_x_projected','velocity projected on XY', 'body',add_horizontal_line = 0)
    except:
        continue
    # exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi')
    # exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi')

    # plot_and_save_movie_prop(exp,yprop = 'mean_front_phi',wing_body = 'mean_wing')
    # plot_and_save_movie_prop(exp,yprop = 'mean_mean_phi',wing_body = 'mean_wing')

